## Scraping Wikipedia Natural Disasters
Follow this step-by-step workflow to collect, clean, and visualise a dataset pulled directly from Wikipedia.

### What you will learn
- Build polite HTTP requests with rotating user agents.
- Identify the right tables on a MediaWiki page before scraping.
- Convert raw HTML rows into a tidy pandas DataFrame.
- Clean numeric ranges, remove footnotes, and fix data types.
- Explore the resulting dataset with plots and a simple map.

### Step 0 — Import the libraries
We bring in web-scraping helpers, data wrangling tools, and visualisation packages that we will use throughout the notebook.

In [ ]:
# Import libraries for HTTP requests, HTML parsing, and randomization
import requests  # For sending HTTP requests to fetch web pages
from bs4 import BeautifulSoup  # For parsing HTML content
import random  # For selecting random user agents

# Import libraries for data manipulation, regular expressions, and string handling
import pandas as pd  # For data manipulation and creating DataFrames
import re  # For regular expressions to clean text
from io import StringIO  # For reading HTML strings as file-like objects

# Import libraries for visualization, mapping, and geocoding
import folium  # For creating interactive maps
import seaborn as sns  # For statistical data visualization
import matplotlib.pyplot as plt  # For plotting graphs
from geopy.geocoders import Nominatim, Photon  # For geocoding locations
from geopy.extra.rate_limiter import RateLimiter  # For rate-limiting geocoding requests

### Step 1 — Rotate user agents
Websites can respond differently depending on the browser profile they think is visiting. Rotating through a small list of realistic user agents helps keep our requests polite and less predictable.

In [ ]:
# Define a list of user agents to rotate through, mimicking different browsers
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/57.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/16.16299',
]

### Step 2 — Helper to pick a user agent
Each request will call this function so we do not reuse the same header repeatedly.

In [ ]:
# Define a function to randomly select a user agent from the list
def get_random_user_agent():
    """Return a random user-agent header for outgoing HTTP requests."""
    return random.choice(user_agents)

# Print an example of a selected user agent
print(f"Using User-Agent: {get_random_user_agent()}")

### Step 3 — Target URL
We will scrape the live Wikipedia article so the dataset stays current. Should the structure change, you can re-run the notebook to refresh the data.

In [ ]:
# Set the URL of the Wikipedia page to scrape
url = "https://en.wikipedia.org/wiki/List_of_natural_disasters_by_death_toll"

### Step 4 — Download the page and inspect its tables
1. Send a request with a random user agent and confirm the status code.
2. Parse the HTML with BeautifulSoup so we can navigate the document.
3. Collect only the tables that expose the columns `'Year', 'Death Toll', 'Event', 'Countries Affected', 'Type', 'Date'`.
4. Preview the first few rows before we tidy the data.

In [ ]:
# Attempt to make an HTTP GET request to the URL
try:
    # Send the request with a random user agent and set a timeout
    response = requests.get(url, headers={"User-Agent": get_random_user_agent()}, timeout=30)
    # Raise an exception if the response status is not successful
    response.raise_for_status()
except requests.RequestException as exc:
    # Exit the program if there's a request error
    raise SystemExit(f"Request error: {exc}") from exc

# Print the HTTP status code and the size of the downloaded content
print(f"HTTP status: {response.status_code}")
print(f"Downloaded {len(response.content):,} bytes of HTML")

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find all tables with the specific classes for collapsible sortable tables
candidate_tables = soup.select("table.wikitable.sortable.mw-collapsible")
# Alternative way: candidate_tables = soup.find_all("table", class_="wikitable sortable mw-collapsible")

# Check if any tables were found, otherwise exit
if not candidate_tables:
    raise SystemExit("No collapsible wikitable tables found. Check if the page layout has changed.")

# Print the number of candidate tables found
print(f"Found {len(candidate_tables)} collapsible sortable tables on the page")

### Step 5 — Extract the rows we care about
Two collapsible tables share the same structure (years 1900–2000 and 2001–present). We will loop through both, normalise their headers, clean each cell (removing footnotes and non-breaking spaces), and combine the results into a single DataFrame that we also persist to disk for reuse.

In [ ]:
# Define the required columns for the dataset
REQUIRED_COLUMNS = ["Year", "Death Toll", "Event", "Countries Affected", "Type", "Date"]

# Dictionary to map various column name variations to standardized names
COLUMN_ALIASES = {
    "death toll": "Death Toll",
    "death tolls": "Death Toll",
    "deaths": "Death Toll",
    "estimated deaths": "Death Toll",
    "total deaths": "Death Toll",
    "event": "Event",
    "event name": "Event",
    "disaster": "Event",
    "natural disaster": "Event",
    "incident": "Event",
    "countries affected": "Countries Affected",
    "country": "Countries Affected",
    "country or region": "Countries Affected",
    "countries": "Countries Affected",
    "countries or regions": "Countries Affected",
    "countries and regions": "Countries Affected",
    "location": "Countries Affected",
    "locations": "Countries Affected",
    "area affected": "Countries Affected",
    "region": "Countries Affected",
    "regions": "Countries Affected",
    "type": "Type",
    "types": "Type",
    "disaster type": "Type",
    "type of disaster": "Type",
    "natural disaster type": "Type",
    "date": "Date",
    "dates": "Date",
    "year": "Year",
    "years": "Year",
}

# Compile regular expressions for cleaning text
CITATION_PATTERN = re.compile(r"\[[^\]]+\]")  # Pattern to remove citations like [1]
WHITESPACE_PATTERN = re.compile(r"\s+")  # Pattern to normalize whitespace

# Function to clean strings by removing citations and normalizing whitespace
def clean_string(value):
    """Return a citation-free, whitespace-normalised string."""
    if pd.isna(value):
        return ""
    text = str(value)
    text = CITATION_PATTERN.sub("", text)  # Remove citations
    text = text.replace("\xa0", " ")  # Replace non-breaking spaces
    text = text.strip()  # Strip leading/trailing whitespace
    return WHITESPACE_PATTERN.sub(" ", text)  # Normalize internal whitespace

# Function to standardize column names based on aliases and patterns
def standardise_column(label):
    if isinstance(label, tuple):
        label = " ".join(str(part) for part in label if part and not pd.isna(part))
    cleaned = clean_string(label)
    slug = re.sub(r"[^a-z0-9]+", " ", cleaned.lower()).strip()
    if slug in COLUMN_ALIASES:
        return COLUMN_ALIASES[slug]
    if "death" in slug and ("toll" in slug or "deaths" in slug):
        return "Death Toll"
    if "event" in slug or "disaster" in slug or "incident" in slug:
        return "Event"
    if "type" in slug:
        return "Type"
    if "country" in slug or "location" in slug or "region" in slug or "area" in slug:
        return "Countries Affected"
    if "date" in slug:
        return "Date"
    if "year" in slug:
        return "Year"
    return cleaned

# List to hold DataFrames from each table
tables_dfs = []
matched_tables = 0

# Loop through each candidate table
for table in candidate_tables:
    # Parse the table into a DataFrame using pandas
    parsed_frames = pd.read_html(StringIO(str(table)), flavor="bs4", header=0)
    if not parsed_frames:
        continue
    table_df = parsed_frames[0]
    # Standardize column names
    table_df.columns = [standardise_column(col) for col in table_df.columns]
    table_df = table_df.loc[:, ~table_df.columns.duplicated()]  # Remove duplicate columns

    # Handle missing columns by inferring or defaulting
    if "Year" not in table_df.columns and "Date" in table_df.columns:
        table_df["Year"] = table_df["Date"]
    if "Date" not in table_df.columns and "Year" in table_df.columns:
        table_df["Date"] = table_df["Year"]
    if "Type" not in table_df.columns:
        table_df["Type"] = "Unknown"
    if "Countries Affected" not in table_df.columns:
        table_df["Countries Affected"] = "Unknown"

    # Check if all required columns are present
    required_missing = set(REQUIRED_COLUMNS) - set(table_df.columns)
    if required_missing:
        continue

    # Select only the required columns
    table_df = table_df[REQUIRED_COLUMNS].copy()
    # Clean all cells in the DataFrame
    table_df = table_df.apply(lambda x: x.apply(clean_string)).replace("", pd.NA)
    # Forward fill missing values for certain columns
    table_df["Countries Affected"] = table_df["Countries Affected"].ffill().fillna("Unknown")
    table_df["Type"] = table_df["Type"].ffill().fillna("Unknown")
    # Remove header rows that might have been parsed as data
    header_mask = table_df["Event"].fillna("").str.lower().eq("event")
    table_df = table_df[~header_mask]

    if table_df.empty:
        continue

    tables_dfs.append(table_df)
    matched_tables += 1

# If no tables were processed, raise an error
if not tables_dfs:
    raise ValueError(
        "No rows extracted after header normalisation. Inspect the page to confirm the expected columns are still present."
    )

# Concatenate all table DataFrames into one
df = pd.concat(tables_dfs, ignore_index=True)
# Extract year as integer from strings
df["Year"] = df["Year"].astype(str).str.extract(r"(\d{3,4})")
# Drop rows with missing Year or Event
df = df.dropna(subset=["Year", "Event"]).reset_index(drop=True)

# Print summary of extraction
print(f"Matched {matched_tables} tables and collected {len(df):,} rows")

# Save the raw DataFrame to CSV
df.to_csv("natural_disasters.csv", index=False, encoding="utf-8")
print("Saved 'natural_disasters.csv' with the raw scrape.")
# Display the first few rows
display(df.head())

## Step 6 — Clean and explore the dataset

### Convert the death toll into comparable integers
Many rows record a range such as `6,000–9,000`. We will keep the lower bound so we can sort and chart consistently.

In [ ]:
# Define a function to extract the lower bound of death toll ranges
def extract_lower_bound(death_toll: str) -> int:
    """Return the first numeric value found in a death toll string."""
    match = re.search(r"(\d+(?:,\d+)?)", str(death_toll))  # Search for numbers with optional commas
    return int(match.group(1).replace(",", "")) if match else 0  # Convert to int, remove commas

# Apply the function to the Death Toll column
df["Death Toll"] = df["Death Toll"].apply(extract_lower_bound)
# Display the updated DataFrame head
display(df.head())

### Inspect the DataFrame shape
Confirm the number of rows (disasters) and columns recorded.

In [ ]:
# Get and display the shape of the DataFrame (rows, columns)
df.shape

### Column names
Double-check that our cleaned headers are the ones we expect.

In [ ]:
# Display the column names of the DataFrame
df.columns

### Summary statistics
A quick numerical overview highlights the distribution of the cleaned death toll values.

In [ ]:
# Generate and display descriptive statistics for the DataFrame
df.describe()

### Check for missing values
Missing data may signal parsing issues or follow-up cleaning tasks.

In [ ]:
# Count and display the number of missing values per column
df.isnull().sum()

### Data types
Verify each column has the expected Python type before visualising.

In [ ]:
# Display the data types of each column
df.dtypes

### Enforce numeric dtypes
Casting to integers makes sure pandas treats years and death tolls as numbers, not strings.

In [ ]:
# Convert Year to integer type
df['Year'] = df['Year'].astype(int)
# Convert Date to datetime type, coercing errors
df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%d", errors='coerce')
# Display updated data types
df.dtypes

### Unique countries affected
How many distinct countries or regions appear in the dataset?

In [ ]:
# Count unique values in 'Countries Affected' column
unique_countries_count = df['Countries Affected'].nunique()
# Print the count
print(f"{unique_countries_count} unique countries or regions recorded.")

### Unique disaster types
A quick count shows the breadth of disaster categories recorded on the page.

In [ ]:
# Count unique values in 'Type' column
unique_disaster_types_count = df['Type'].nunique()
# Print the count
print(f"{unique_disaster_types_count} distinct disaster types captured.")

In [ ]:
# Set pandas display options to show full table (optional, can be slow for large data)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Display the full DataFrame
df

### Optional: display the full table
Expand pandas' display settings if you want to inspect every column and row inline. Be cautious—large tables can slow down the notebook.

## Step 7 — Visualise the impact

### Year vs. death toll
Plotting each disaster as a point shows temporal clusters and highlights extreme events by type.

In [ ]:
# Create a scatter plot of Year vs Death Toll, colored by Type
plt.figure(figsize=(12, 6))
sns.scatterplot(x='Year', y='Death Toll', data=df, hue='Type', style='Type', palette='Set1', s=100)
plt.title('Scatter Plot of Year vs. Death Toll by Type')
plt.xlabel('Year')
plt.ylabel('Death Toll')
plt.legend(title='Type', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

### Total deaths by disaster type
Aggregating by type reveals which kinds of disasters have historically been most deadly.

In [ ]:
# Group by Type and sum Death Toll, then sort descending
total_death_by_type = df.groupby('Type')['Death Toll'].sum().reset_index().sort_values('Death Toll', ascending=False)
# Create a bar plot
plt.figure(figsize=(12, 6))
sns.barplot(x='Type', y='Death Toll', data=total_death_by_type, palette='Set1', hue='Type', legend=False)
plt.title('Total Death Toll by Type')
plt.xlabel('Type')
plt.ylabel('Total Death Toll')
plt.xticks(rotation=90)
plt.show()

### Total deaths by country or region
Summing per location highlights where disasters have had the heaviest tolls.

In [ ]:
# Group by Countries Affected and sum Death Toll, then sort descending
total_death_by_type = df.groupby('Countries Affected')['Death Toll'].sum().reset_index().sort_values('Death Toll', ascending=False)
# Create a bar plot
plt.figure(figsize=(12, 6))
sns.barplot(x='Countries Affected', y='Death Toll', data=total_death_by_type, palette='Set1', hue='Countries Affected', legend=False)
plt.title('Total Death Toll by Countries affected')
plt.xlabel('Countries affected')
plt.ylabel('Total Death Toll')
plt.xticks(rotation=90)
plt.show()

### Recent 20-year view
Filter to the last two decades to surface modern disaster hotspots.

In [ ]:
# Filter DataFrame to years after 2003, group by Countries Affected, sum Death Toll, sort descending
total_death_by_type = df[df['Year']>2003].groupby('Countries Affected')['Death Toll'].sum().reset_index().sort_values('Death Toll', ascending=False)
# Create a bar plot
plt.figure(figsize=(12, 6))
sns.barplot(x='Countries Affected', y='Death Toll', data=total_death_by_type, palette='Set1', hue='Countries Affected', legend=False)
plt.title('Total Death Toll by Countries affected')
plt.xlabel('Countries affected')
plt.ylabel('Total Death Toll')
plt.xticks(rotation=90)
plt.show()

### Disaster impact over time
Line charts help reveal trajectory trends and make it easier to spot slow-moving crises.

In [ ]:
# Create a line plot of Year vs Death Toll, colored by Type
sns.relplot(
    data=df, kind="line",
    x="Year", y="Death Toll",
    hue="Type")

In [ ]:
# Create a line plot excluding common types like Earthquake and Earthquake, Tsunami
sns.relplot(
    data=df[~df['Type'].isin(['Earthquake', 'Earthquake, Tsunami'])], kind="line",
    x="Year", y="Death Toll",
    hue="Type")

### Top 10 deadliest events
Sort the dataset to spotlight the most catastrophic disasters on record.

In [ ]:
# Get the top 10 events by Death Toll
top_events = df.nlargest(10, 'Death Toll')
# Create a horizontal bar plot
plt.figure(figsize=(10, 6))
sns.barplot(x='Death Toll', y='Event', data=top_events, palette='viridis', hue='Event', legend=False)
plt.title('Top 10 Events with Highest Death Toll')
plt.xlabel('Death Toll')
plt.ylabel('Event')
plt.show()

### Interactive map of affected countries
Geocode each country and plot markers. This step uses rate-limited lookups (update the user agent with your contact information) and may take a couple of minutes to run.

In [ ]:
# Set a user agent for geocoding services (required by Nominatim)
USER_AGENT = "DisasterGeoMapper/1.0 (contact: your_email@example.com)"

# Create a base folium map centered at (0,0) with zoom level 2
m = folium.Map(location=[0, 0], zoom_start=2)

# Initialize geocoders with user agent and timeout
nominatim = Nominatim(user_agent=USER_AGENT, timeout=10)
photon = Photon(user_agent=USER_AGENT, timeout=10)

# Apply rate limiting to geocoding functions
nominatim_geocode = RateLimiter(nominatim.geocode, min_delay_seconds=1, swallow_exceptions=True)
photon_geocode = RateLimiter(photon.geocode, min_delay_seconds=1, swallow_exceptions=True)

# Cache for geocoded locations to avoid repeated requests
location_cache = {}

# Function to geocode a country name with caching and fallback
def geocode_country(name: str):
    """Safely geocode a country name with caching and fallback between Nominatim and Photon."""
    if name in location_cache:
        return location_cache[name]
    
    location = nominatim_geocode(name)
    if location is None:
        location = photon_geocode(name)
    
    location_cache[name] = location
    return location

# Dictionary to aggregate disasters by country
disasters_by_country = {}

# Loop through each row in the DataFrame
for _, row in df.iterrows():
    year = row['Year']
    death_toll = row['Death Toll']
    countries = row['Countries Affected']
    event = row['Event']

    # Split countries if multiple are listed
    for country in countries.split(','):
        country = country.strip()
        if not country or country.lower() in {"various", "unknown"}:
            continue
        
        # Geocode the country
        location = geocode_country(country)
        if location is None:
            continue
        
        latitude, longitude = location.latitude, location.longitude
        # Create tooltip with disaster details
        tooltip = f"Year: {year}<br>Death Toll: {death_toll}<br>Country: {country}<br>Event: {event}"
        # Append to list for this country
        disasters_by_country.setdefault(country, []).append((latitude, longitude, tooltip))

# Add markers to the map, averaging locations per country
for country, disasters in disasters_by_country.items():
    country_latitude = sum(lat for lat, lon, _ in disasters) / len(disasters)
    country_longitude = sum(lon for lat, lon, _ in disasters) / len(disasters)
    country_tooltip = "<br>".join(tooltip for _, _, tooltip in disasters)
    folium.Marker([country_latitude, country_longitude], tooltip=country_tooltip).add_to(m)

In [ ]:
# Display the interactive map in the notebook
m

In [ ]:
# Save the map to an HTML file
m.save('impact_by_country_map.html')